#### Libraries

In [ ]:
import pandas as pd
import numpy as np
import os
import re
# import random 
# from shutil import copyfile
import shutil
# #import pydicom #as dicom
import cv2
from PIL import Image

import zipfile
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# # web scraping
import requests
#from bs4 import BeautifulSoup

from progressbar import ProgressBar

# import re
import time
# from random import randrange
# from datetime import datetime

from image_data import extract_images

import matplotlib.pyplot as plt
# import winsound
# frequency = 500  
# duration = 800  # Set Duration To 800 ms == 0.8 second

In [ ]:
print("Pandas", pd.__version__)
import selenium
print("selenium", selenium.__version__)
print("requests", requests.__version__)

#### Functions

In [ ]:
def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')
    
def remove_html_tags(text):
    """Function to remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


#### Parameters

In [ ]:
# set save path directory
SAVE_PATH = 'data'

# create data, video, and image folders, if they do not exist
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('data/video'):
    os.makedirs('data/video')
if not os.path.exists('data/image'):
    os.makedirs('data/image')
    
# setting chrome driver
chromedriver = "utils/chromedriver.exe" 
os.environ["webdriver.chrome.driver"] = chromedriver
chrome_options = Options()
chrome_options.add_argument("--headless")

# setting global vars
VIDEO_PATH = 'data/video/'
IMAGE_PATH = 'data/image/'

#### Read the Metadata File

In [ ]:
metadata = pd.read_csv('utils/video_metadata.csv', sep=',', encoding='latin1')
print(metadata.shape)
metadata.head(2)

# 1. Get Ultrasound Videos

## 1.1. ButterflyNetwork

In [ ]:
# zip file url
butterfly_url = metadata[metadata.source == 'Butterfly'].url.unique()[0]
print('...Downloading ButterflyNetwork zip file...')

# simulatting button click to download the zip file
browser = webdriver.Chrome(chromedriver) #, options=chrome_options)
browser.get(butterfly_url)
# Download button sometimes doesn't load in time to click
# uncomment time.sleep(5) if such error occurring
# time.sleep(5)
browser.find_element_by_class_name('btn-primary').click()

# path to the downloaded zip file
zip_file_path = os.path.join(get_download_path(), 'COVID Clinical Gallery Butterfly Network.zip')  # will be downloaded in your Download folder!

# wait till the zip file is downloaded
while not os.path.exists(zip_file_path):
    time.sleep(1)

# create butterfly folder under video folder, if it does not exist
if not os.path.exists('data/tmp/Butterfly'):
    os.makedirs('data/tmp/Butterfly')

print('...Extracting the video files...')
# extract the downloaded zip file and remove the zip file after extraction
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(os.path.join(SAVE_PATH, 'tmp/Butterfly'))

# copy files from subfolders to the video folder
for root, dirs, files in os.walk('data/tmp/Butterfly'):  
    for file in files:
        path_file = os.path.join(root,file)
        shutil.copy2(path_file, 'data/video') 

# renaming extracted files to their ids
progress = ProgressBar() 
for root, dirs, files in os.walk('data/video'):  
    for file in progress(files):
        path_file = os.path.join(root,file)
        file_id = metadata[metadata.filename == file].id.values[0] + '.mp4'
        # rename the file to its id
        os.rename(path_file, os.path.join(root,file_id))

print('=== ButterflyNetwork video files extraction done! ===')        
        
# delete the tmp folder and its contents
shutil.rmtree('data/tmp')
        
# remove the zip file
os.remove(zip_file_path)

## 1.2. GrepMed

In [ ]:
print('...Extracting the video files...')
grepmed_df = metadata[metadata.source == 'GrepMed']

progress = ProgressBar(max_value=grepmed_df.shape[0]) 
for idx, row in progress(grepmed_df.iterrows()):
    filename = row.id + '.' + row.filetype
    # write the video file to disk
    vid = requests.get(row.url).content
    with open(os.path.join('data/video/', filename), 'wb') as handler:
        handler.write(vid)
print('=== GrepMed video files extraction done! ===')        

## 1.3. LITFL

In [ ]:
print('...Extracting the video files...')
litfl_df = metadata[metadata.source == 'Litfl']

progress = ProgressBar(max_value=litfl_df.shape[0]) 
for idx, row in progress(litfl_df.iterrows()):
    filename = row.id + '.' + row.filetype
    # write the video file to disk
    vid = requests.get(row.url).content
    with open(os.path.join('data/video/', filename), 'wb') as handler:
        handler.write(vid)
print('=== LITFL video files extraction done! ===')        

## 1.4. The POCUS Atlas

In [ ]:
print('...Extracting the video files...')
pocus_df = metadata[metadata.source == 'PocusAtlas']

progress = ProgressBar(max_value=pocus_df.shape[0]) 
for idx, row in progress(pocus_df.iterrows()):
    filename = row.id + '.' + row.filetype
    # write the video file to disk
    vid = requests.get(row.url).content
    with open(os.path.join('data/video/', filename), 'wb') as handler:
        handler.write(vid)
print('=== THEPocusAtlas video files extraction done! ===')        

# 2. Video Preprocessing

## 2.1. Getting Video Files Properties

In [ ]:
vid_files = os.listdir(VIDEO_PATH)

progress = ProgressBar(max_value=metadata.shape[0]) 
with open('utils/video_files_properties.csv', 'w') as f:
    # write the file header
    f.write('filename,framerate,width,height,frame_count,duration_secs\n')
    
    # loop over the video files and get their properties
    for vid in progress(vid_files):
        vid_filename = VIDEO_PATH + str(vid)
        file_type = vid.split('.')[-1]
        
        # get video file properties
        cv2video = cv2.VideoCapture(vid_filename)
        height = cv2video.get(cv2.CAP_PROP_FRAME_HEIGHT)
        width  = cv2video.get(cv2.CAP_PROP_FRAME_WIDTH) 
        frame_rate = round(cv2video.get(cv2.CAP_PROP_FPS), 2)
        
        if file_type == 'mp4':
            frame_count = cv2video.get(cv2.CAP_PROP_FRAME_COUNT) 
            duration = round((frame_count / frame_rate), 2)
        elif file_type == 'gif':
            frame_count = round(Image.open(vid_filename).n_frames) #round((duration * frame_rate ), 0)
            duration = round((frame_count / frame_rate), 2)
            #duration = round((1000 / Image.open(vid_filename).info['duration']) , 2)

        # write video properties to the file
        line_to_write = str(vid) + ',' + str(frame_rate) + ',' + str(width) + ',' + str(height) + ',' + str(frame_count) + ',' + str(duration) + '\n'
        f.write(line_to_write)

## 2.2. Video Preprocessing

In [ ]:
VIDEO_PROCESSED_OUT = 'data/video/processed/'
VIDEO_CROPPED_OUT = 'data/video/processed/cropped/'

# create processed and cropped folder if they don't already exist
if not os.path.exists('data/video/processed/'):
    os.makedirs('data/video/processed/')
if not os.path.exists('data/video/processed/cropped'):
    os.makedirs('data/video/processed/cropped')

### 2.2.1. Inital Cropping

In [ ]:
# read cropping metadata file
vid_crp_metadata = pd.read_csv('utils/video_cropping_metadata.csv', sep=',', encoding='latin1')
print(vid_crp_metadata.shape)
vid_crp_metadata.head(2)

In [ ]:
progress = ProgressBar(max_value=vid_crp_metadata.shape[0])

for idx, row in progress(vid_crp_metadata.iterrows()):
    vid_arr = []  # array to store frames of a video file
    
    filename = row.filename
    file_label = filename.split('_')[-1].split('.')[0] # label of the video file
    
    cap = cv2.VideoCapture(os.path.join(VIDEO_PATH, filename))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
    dim = (width, height) # dimension of the original file
    
    if pd.isna(row.x1_w_y1_h): # square cropping
        DEL_UPPER = int(row.del_upper) # to remove top
        WIDTH_RATE = float(row.width_rate) # to remove sides e.g. the meter
        
        width_border = int(width * WIDTH_RATE)
        width_box = int(width - (2 * width_border)) 
        if width_box + DEL_UPPER > height:
            width_box = int(height - DEL_UPPER)
            width_border = int( (width / 2) - (width_box / 2))

        while(True):
            ret, frame = cap.read()

            if not ret:
                break

            # crop
            frame = frame[DEL_UPPER:width_box + DEL_UPPER, width_border:width_box + width_border]

            frame = np.asarray(frame).astype(np.uint8)
            vid_arr.append(frame)

    else: # crop using (x1,y1) and (x2, y2). The output will not be necessarily a square file
        X1 = int(row.x1_w_y1_h.split(',')[0].replace('(', ''))
        W = int(row.x1_w_y1_h.split(',')[1].strip())
        Y1 = int(row.x1_w_y1_h.split(',')[2].strip())
        H = int(row.x1_w_y1_h.split(',')[3].replace(')', '').strip())

        while(True):
            ret, frame = cap.read()

            if not ret:
                break

            # crop
            frame = frame[Y1:Y1 + H, X1:X1 + W]

            frame = np.asarray(frame).astype(np.uint8)
            vid_arr.append(frame)

    vid_arr = np.asarray(vid_arr)
    prc_dim = vid_arr.shape[1:3] # dimension of the cropped file
    prc_dim = (prc_dim[1], prc_dim[0])

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(os.path.join(VIDEO_CROPPED_OUT + filename.split('.')[0] + '_prc.avi'), fourcc, 20.0, tuple(prc_dim))

    for frame in vid_arr:
        out.write(frame.astype("uint8"))

    vid_crp_metadata.iloc[idx, vid_crp_metadata.columns.get_loc('crp_width')] = prc_dim[1]
    vid_crp_metadata.iloc[idx, vid_crp_metadata.columns.get_loc('crp_height')] = prc_dim[0]

    cap.release()
    out.release()
    cv2.destroyAllWindows()

vid_crp_metadata.to_csv('utils/video_cropping_metadata.csv', index=None)

print('Initial cropping done...')

# 3. Extract Ultrasound Images from Videos

In [ ]:
# VID_CLASS = ['COVID', 'Pneumonia', 'Normal']
# VID_SOURCE = ['Butterfly', 'GrepMed', 'LITFL', 'PocusAtlas']
# PROB_TYPE = ['convex', 'linear']

#### Read video properties

In [ ]:
vid_prop_df = pd.read_csv('utils/video_files_properties.csv')

# merge with the video meta data file 
vid_prop_df.filename = vid_prop_df.filename.astype(str)
vid_prop_df.filename = vid_prop_df.filename.str.strip()

metadata['filename2'] = metadata.id + '.' + metadata.filetype
metadata.filename2 = metadata.filename2.astype(str)
metadata.filename2 = metadata.filename2.str.strip()

vid_prop_df = pd.merge(vid_prop_df, metadata[['filename2', 'source', 'probe', 'class']], left_on='filename', right_on='filename2', how='left').drop('filename2', axis=1)

del metadata['filename2']
vid_prop_df.head(2)

## 3.1. Extract frames from cropped video files

In [ ]:
IMAGE_CROPPED_OUT = 'data/image/cropped/'
#IMAGE_INPAINT_OUT = 'data/image/inpainted/'
IMAGE_MASK_OUT = 'data/mask/'

# create cropped and inpainted image folders and the mask folder if they don't already exist
if not os.path.exists(IMAGE_CROPPED_OUT):
    os.makedirs(IMAGE_CROPPED_OUT)
# if not os.path.exists(IMAGE_INPAINT_OUT):
#     os.makedirs(IMAGE_INPAINT_OUT)
if not os.path.exists(IMAGE_MASK_OUT):
    os.makedirs(IMAGE_MASK_OUT)

In [ ]:
extract_images(video_path= VIDEO_CROPPED_OUT, image_path=IMAGE_CROPPED_OUT, cropped=True)

### 3.1.1. (Optional) Extracting frames from cropped ultrasouund video files using a parameter set as filter
* You can extract images using the follwoing parameters:
    * maximum number of frames to be extracted from each video file
    * extracting a targetted set of classes from ['COVID', 'Pneumonia', 'Normal']
    * extracting a targetted set of data sources from ['Butterfly', 'GrepMed', 'LITFL', 'PocusAtlas']
    * extracting a targetted set of probes from ['convex', 'linear']

In [ ]:
#extract_images(video_path= VIDEO_CROPPED_OUT, image_path=IMAGE_CROPPED_OUT, cropped=True, 
#                max_frames=10, 
#                target_class=['COVID', 'Pneumonia', 'Normal'],
#                target_source=['Butterfly', 'GrepMed', 'LITFL', 'PocusAtlas'],
#                target_probe=['convex', 'linear']))

## 3.2. Preprocessing Images

#### Read image preprocessing metadata

In [ ]:
image_prc_df = pd.read_csv('utils/mask_metadata.csv')

print(image_prc_df.shape)
image_prc_df.head(2)

### 3.2.1. Removing frames with artifacts on the ROI
Some frames of the following files need to be deleted as the moving pointer is on ROI:
* 2_butterfly_covid.mp4
* 6_butterfly_covid.mp4
* 16_butterfly_covid.mp4
* 20_butterfly_normal.mp4
* 22_butterfly_covid.mp4
* 25_grepmed_pneumonia.mp4


* __Initial total number of frames:__ 11,974
* __Total number of frames after removal:__ 8,549

In [ ]:
progress = ProgressBar(max_value=image_prc_df[~pd.isna(image_prc_df.delete_frames_from_to)].shape[0])

for idx, row in progress(image_prc_df[~pd.isna(image_prc_df.delete_frames_from_to)].iterrows()):
    frames_to_delete = row.delete_frames_from_to.strip().split(',')
    frame_name_main = row.mask_main_filename.split('.')[0].split('_frame')[0]
    
    for frames in frames_to_delete:
        from_frame = int(frames.split('-')[0])
        to_frame = int(frames.split('-')[1]) + 1
        
        # delete frames with moving part on the roi
        for i in range(from_frame, to_frame):
            file_to_remove = IMAGE_CROPPED_OUT + frame_name_main + '_frame' + str(i) + '.jpg'
            os.remove(file_to_remove)

print("=== Files removed! ===")

### 3.2.2. Applying masks

In [ ]:
CLEAN_IMAGE_OUT = 'data/image/clean/'
CLEAN_VIDEO_OUT = 'data/video/clean/'

# create clean image and video folders if they don't already exist
if not os.path.exists(CLEAN_IMAGE_OUT):
    os.makedirs(CLEAN_IMAGE_OUT)
if not os.path.exists(CLEAN_VIDEO_OUT):
    os.makedirs(CLEAN_VIDEO_OUT)

In [ ]:
image_prc_df.head(2)

In [ ]:
def zero_pad_array(arr, pad=5):
    if len(arr.shape) == 3:
        padded_arr = np.zeros((arr.shape[0]+2*pad, arr.shape[1]+2*pad, arr.shape[2]), dtype=np.uint8)
        padded_arr[pad:pad + arr.shape[0], pad:pad + arr.shape[1], :] = arr
    else:
        padded_arr = np.zeros((arr.shape[0]+2*pad, arr.shape[1]+2*pad), dtype=np.uint8)
        padded_arr[pad:pad + arr.shape[0], pad:pad + arr.shape[1]] = arr
    return padded_arr
        

def frame_inpainting(frame_dict, mask, default_mask=0, kernel_size=(5,5), method='telea', pad=5):
    '''
    The function performs inpainting on frames using the created masks
    
    - frame_dict: dict of frames from video, indexed by frame number
    - mask: (h, w, 1) array if single mask, else dict of such arrays
        indexed by frame number
    - default_mask: index for mask to be used as default, for frames
        without specific mask (if mask is not constant across frames)
    - kernel_size: Size of patch used to perform inpainting
    - method: one of 'ns' (navier-stokes) or 'telea' - telea usually works better
    '''
    # Dilate mask make sure it covers enough of the ROI to be masked
    kernel = np.ones(kernel_size, np.uint8)
    if type(mask) is not dict:
        mask = {default_mask: mask}
    masks_processed = {key:cv2.dilate(zero_pad_array(m, pad=pad), kernel, iterations=1) for key, m in mask.items()}
    
    method_dict = {'ns':cv2.INPAINT_NS, 'telea':cv2.INPAINT_TELEA}
    
    frames_inpainted = {}
    for key, frame in frame_dict.items():
        if key in masks_processed:
            #print(frame.shape, masks_processed[key].shape)
            frames_inpainted[key] = cv2.inpaint(zero_pad_array(frame, pad=pad), masks_processed[key], 3, method_dict[method])[pad:-pad, pad:-pad, :]
        else: # default mask
            frames_inpainted[key] = cv2.inpaint(zero_pad_array(frame, pad=pad), masks_processed[default_mask], 3, method_dict[method])[pad:-pad, pad:-pad, :]


    return frames_inpainted

In [ ]:
progress = ProgressBar(max_value=image_prc_df.shape[0])

for idx, row in progress(image_prc_df.iterrows()):     # .iloc[1:2, :]   .iloc[27:28, :]
    # get the main token of the filename
    if row.probe == 'Convex':
        filename_main = row.filename.split('.')[0] + '_prc_convex'
    elif row.probe == 'Linear':
        filename_main = row.filename.split('.')[0] + '_prc_linear'
        
    if row.tight_inpainting == 'yes':
        # objects close to ROI, avoid bleeding while inpainting
        inpainting_kernel_size = (1,1)
    else:
        # no objects close to ROI, more effective inpainting
        inpainting_kernel_size = (5,5)

    # check if the cropped frames need cleaning
    if row.need_mask_after_crop == 'no':
        frames = {}
        
        # 1. no clearning, copy cropped images and rename them to clean folder
        for file in os.listdir(IMAGE_CROPPED_OUT):
            if file.startswith(filename_main):
                #last_part = file.split('_')[-1]
                #last_part = last_part.replace('frame', '_clean_frame')
                new_filename = file.replace('frame', 'clean_frame')
                #print(file, new_filename) #, last_part)
                shutil.copy(IMAGE_CROPPED_OUT + file, CLEAN_IMAGE_OUT + new_filename)

                img = cv2.imread(os.path.join(CLEAN_IMAGE_OUT, new_filename))
                frame_num = int(re.search(r'\d+$', file[:-4]).group())
                frames[frame_num] = img
        
        # make a video out of the clean frames
        keys = list(frames.keys())
        keys.sort()
        clean_vid_frames = [frames[k] for k in keys]

        h, w, layers = clean_vid_frames[0].shape
        size = (w, h)

        out = cv2.VideoWriter(os.path.join(CLEAN_VIDEO_OUT + filename_main + '_clean.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
        for i in range(len(clean_vid_frames)):
            out.write(clean_vid_frames[i])
        out.release()    
        
    # 2. frames need cleaning
    else: 
        # create a dictionary of frames
        frames = {}
        for f in os.listdir(IMAGE_CROPPED_OUT):
            if f.startswith(filename_main):
                img = cv2.imread(os.path.join(IMAGE_CROPPED_OUT, f))
                frame_num = int(re.search(r'\d+$', f[:-4]).group())
                frames[frame_num] = img

        # check if the video file requires multiple masks or a single mask is enough
        if row.need_multiple_masks == 'no':
            mask = cv2.imread(os.path.join(IMAGE_MASK_OUT, filename_main + '_frame0_mask.jpg'))[:,:,0]

            # perform inpainting on frames using a single main mask
            frames_inpainted = frame_inpainting(frames, mask, kernel_size=inpainting_kernel_size)
        else:
            masks = {}
            for f in os.listdir(IMAGE_MASK_OUT):
                if f.startswith(filename_main):
                    img = cv2.imread(os.path.join(IMAGE_MASK_OUT, f))
                    frame_num = int(re.search(r'\d+$', f[:-9]).group())
                    masks[frame_num] = img[:,:,0]

            # perform inpainting on frames using multiple masks
            frames_inpainted = frame_inpainting(frames, masks, default_mask=0, kernel_size=inpainting_kernel_size)

        # write clean frames to the disk
        for key, value in frames_inpainted.items():
            cv2.imwrite(CLEAN_IMAGE_OUT + filename_main + "_clean_frame" + str(key) + ".jpg", value)

        # write clean video to the disk
        keys = list(frames_inpainted.keys())
        keys.sort()
        clean_vid_frames = [frames_inpainted[k] for k in keys]

        h, w, layers = clean_vid_frames[0].shape
        size = (w, h)

        out = cv2.VideoWriter(os.path.join(CLEAN_VIDEO_OUT + filename_main + '_clean.avi'), cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
        for i in range(len(clean_vid_frames)):
            out.write(clean_vid_frames[i])
        out.release()    